In [14]:
import pandas as pd
import csv

# Chargement des datasets

In [10]:
train = pd.read_csv('./input/train.csv')

In [11]:
test = pd.read_csv('./input/test.csv')

UserKey contient les headers de colonnes du fichier User, nous récupérons cette liste. Le fichier étant mal formaté (virgules dans les descriptions, nous préprocessons le ficier pour le mettre sous le format correct.

In [40]:
with open('./input/UserKey.csv', encoding='utf-8') as infile, open('./input/quotedUserKey.csv','w', encoding='utf-8') as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile)
    for line in reader:
        if len(line) > 2:
            temp_line = line[1]
            for i in range(2, len(line)):
                line[1] +=', '+line[i]
            line = line[:2]
        writer.writerow(line)                        

In [42]:
user_key = pd.read_csv('./input/quotedUserKey.csv', header=None)

In [44]:
users = pd.read_csv('./input/users.csv')

le csv 'words' contient des caractères non compatibles avec le format utf-8, nous réenregistront le fichier sous le bon format à l'aide d'un éditeur de texte.

In [48]:
words = pd.read_csv('./input/words.csv')

In [6]:
train.head(10)

,Artist,Track,User,Rating,Time
0,40,179,47994,9,17
1,9,23,8575,58,7
2,46,168,45475,13,16
3,11,153,39508,42,15
4,14,32,11565,54,19
5,31,79,27130,37,11
6,21,48,19623,69,21
7,2,174,47505,28,17
8,12,34,15290,12,8
9,28,73,24151,70,22


In [9]:
test.head(10)

,Artist,Track,User,Time
0,1,6,3475,18
1,6,149,39210,15
2,40,177,47861,17
3,31,79,27413,11
4,26,66,23232,22
5,26,61,22055,22
6,2,68,24799,22
7,25,59,18862,21
8,0,150,38832,15
9,1,6,2963,18


L'objectif de ce challenge est de prédire la valeur de la variable "Rating" sur le jeu de test. Cette variable a une valeur entière comprise entre X-100, comme il s'agit d'une notation, nous pouvons dire que c'est une variable continue et donc quil s'agit d'un problème de régression. De plus la métrique  d'évaluation est RMSE (Root Mean Squared Error) ce qui est une métrique de régression. L'objectif est d'obtenir un csv conenant les prédictions des évaluations pour le jeu de test.

In [53]:
user_key.head(2)

,0,1
0,RESPID,User id
1,GENDER,Male/Female


In [52]:
users.head(2)

,RESPID,GENDER,AGE,WORKING,REGION,MUSIC,LIST_OWN,LIST_BACK,Q1,Q2,...,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19
0,36927,Female,60.0,Other,South,Music is important to me but not necessarily m...,1 hour,NaN,49.0,50.0,...,50.0,50.0,71.0,52.0,71.0,9.0,7.0,72.0,49.0,26.0
1,3566,Female,36.0,Full-time housewife / househusband,South,Music is important to me but not necessarily m...,1 hour,1 hour,55.0,55.0,...,12.0,65.0,65.0,80.0,79.0,51.0,31.0,68.0,54.0,33.0


In [54]:
words.head(10)

,Artist,User,HEARD_OF,OWN_ARTIST_MUSIC,LIKE_ARTIST,Uninspired,Sophisticated,Aggressive,Edgy,Sociable,...,Unoriginal,Dated,Iconic,Unapproachable,Classic,Playful,Arrogant,Warm,Soulful,Unnamed: 87
0,47,45969,Heard of,NaN,NaN,NaN,0.0,NaN,0,0.0,...,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0,0.0,NaN
1,35,29118,Never heard of,NaN,NaN,0.0,NaN,0.0,0,NaN,...,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0,NaN,NaN
2,14,31544,Heard of,NaN,NaN,0.0,NaN,0.0,0,NaN,...,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0,NaN,NaN
3,23,18085,Never heard of,NaN,NaN,NaN,NaN,0.0,0,NaN,...,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0,NaN,NaN
4,23,18084,Never heard of,NaN,NaN,NaN,NaN,0.0,0,NaN,...,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0,NaN,NaN
5,23,18082,Never heard of,NaN,NaN,NaN,NaN,0.0,0,NaN,...,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0,NaN,NaN
6,23,18080,Never heard of,NaN,NaN,NaN,NaN,0.0,1,NaN,...,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0,NaN,NaN
7,14,31541,Heard of,NaN,NaN,0.0,NaN,0.0,0,NaN,...,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0,NaN,NaN
8,46,43802,Heard of and listened to music EVER,Own none of their music,25.0,NaN,NaN,0.0,0,NaN,...,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0,NaN,NaN
9,23,18077,Never heard of,NaN,NaN,NaN,NaN,0.0,0,NaN,...,1.0,1.0,NaN,1.0,0.0,0.0,0.0,0,NaN,NaN


# Exploration de données

# Constitution du dataset